In [ ]:
import requests
import pandas as pd

base_url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": [40.712776, 41.82399, 42.440498, 43.700859, 42.3656347, 40.3496953, 39.9527237, 41.3082138],
    "longitude": [-74.005974, -71.412834, -76.495697, -72.289398, -71.1040018, -74.6597376, -75.1635262, -72.9250518],
    "daily": "temperature_2m_min",
    "hourly": ["temperature_2m", "sunshine_duration"],
    "timezone": "America/Chicago",
    "wind_speed_unit": "mph",
    "temperature_unit": "fahrenheit",
    "precipitation_unit": "inch",
    "start_date": "2026-01-01",
    "end_date": "2026-01-31"
}

response = requests.get(base_url, params=params)
data = response.json()

In [ ]:
all_dfs = []
for i, loc in enumerate(data):
    loc_df = pd.DataFrame({
        'location': i + 1,
        'date': loc['daily']['time'],
        'min_temp_f': loc['daily']['temperature_2m_min']
    })
    all_dfs.append(loc_df)

df = pd.concat(all_dfs, ignore_index=True)
df

In [ ]:
ivy_league_schools = {
    "Harvard University": {"city": "Cambridge", "state": "MA"},
    "Yale University": {"city": "New Haven", "state": "CT"},
    "Princeton University": {"city": "Princeton", "state": "NJ"},
    "Columbia University": {"city": "New York", "state": "NY"},
    "University of Pennsylvania": {"city": "Philadelphia", "state": "PA"},
    "Brown University": {"city": "Providence", "state": "RI"},
    "Dartmouth College": {"city": "Hanover", "state": "NH"},
    "Cornell University": {"city": "Ithaca", "state": "NY"}
}

base_url = "https://api.data.gov/ed/collegescorecard/v1/schools"
all_schools_data = []

for school_name, location in ivy_league_schools.items():
    params = {
        "school.name": school_name,
        "fields": "id,school.name,school.city,school.state,location.lat,location.lon",
        "api_key": "aUYfWIfRdwDrbSfTIm3lTm5M67bWMk1YQNaqz4go"
    }
    
    response = requests.get(base_url, params=params)
    data = response.json()
    
    if data.get('results'):
        all_schools_data.extend(data['results'])
        print(f"Found: {school_name}")
    else:
        print(f"Not found: {school_name}")

schools_df = pd.DataFrame(all_schools_data)
print("\nIvy League Schools Data:")
schools_df

In [ ]:
import requests
import pandas as pd

# Step 1: Get Ivy League school data
ivy_league_schools = {
    "Harvard University": {"city": "Cambridge", "state": "MA"},
    "Yale University": {"city": "New Haven", "state": "CT"},
    "Princeton University": {"city": "Princeton", "state": "NJ"},
    "Columbia University": {"city": "New York", "state": "NY"},
    "University of Pennsylvania": {"city": "Philadelphia", "state": "PA"},
    "Brown University": {"city": "Providence", "state": "RI"},
    "Dartmouth College": {"city": "Hanover", "state": "NH"},
    "Cornell University": {"city": "Ithaca", "state": "NY"}
}

base_url_schools = "https://api.data.gov/ed/collegescorecard/v1/schools"
all_schools_data = []

for school_name, location in ivy_league_schools.items():
    params = {
        "school.name": school_name,
        "fields": "id,school.name,school.city,school.state,location.lat,location.lon",
        "api_key": "aUYfWIfRdwDrbSfTIm3lTm5M67bWMk1YQNaqz4go"
    }
    
    response = requests.get(base_url_schools, params=params)
    data = response.json()
    
    if data.get('results'):
        for result in data['results']:
            if result.get('school.name') == school_name:
                all_schools_data.append(result)
                print(f"✓ Found: {school_name}")
                break

# Create schools DataFrame
schools_df = pd.DataFrame(all_schools_data)
print("\nSchools DataFrame:")
print(schools_df)

# Step 2: Get weather data for each school
base_url_weather = "https://archive-api.open-meteo.com/v1/archive"

all_weather_data = []

for index, school in schools_df.iterrows():
    lat = school['location.lat']
    lon = school['location.lon']
    school_name = school['school.name']
    
    params = {
        "latitude": lat,
        "longitude": lon,
        "daily": "temperature_2m_min",
        "timezone": "America/New_York",
        "temperature_unit": "fahrenheit",
        "start_date": "2026-01-01",
        "end_date": "2026-01-31"
    }
    
    response = requests.get(base_url_weather, params=params)
    weather_data = response.json()
    
    if 'daily' in weather_data:
        # Create a row for each day
        for i, date in enumerate(weather_data['daily']['time']):
            all_weather_data.append({
                'school_name': school_name,
                'latitude': lat,
                'longitude': lon,
                'date': date,
                'temp_min': weather_data['daily']['temperature_2m_min'][i]
            })
        print(f"✓ Weather data retrieved for: {school_name}")

# Create weather DataFrame
weather_df = pd.DataFrame(all_weather_data)

# Step 3: Join the DataFrames
# Merge schools with weather data
combined_df = pd.merge(
    schools_df,
    weather_df,
    left_on=['school.name', 'location.lat', 'location.lon'],
    right_on=['school_name', 'latitude', 'longitude'],
    how='left'
)

# Clean up duplicate columns
combined_df = combined_df.drop(['school_name', 'latitude', 'longitude'], axis=1)

# Display results
print("\n" + "="*80)
print("COMBINED IVY LEAGUE SCHOOLS & WEATHER DATA")
print("="*80)
print(combined_df.head(20))